# Creating Word Embeddings with fasttext
Our tabseparated text corpus:

In [ ]:
! ls -lh parlament_transcriptions_tokenized_de.tsv
! tail -n 1 parlament_transcriptions_tokenized_de.tsv 

Download the precomputed file if needed

In [ ]:
! wget https://files.ifi.uzh.ch/cl/siclemat/lehre/hs19/tm/parlament_transcriptions_tokenized_de.tsv -O parlament_transcriptions_tokenized_de.tsv

## Building word embeddings with fasttext

In [ ]:
! cut -f 3 parlament_transcriptions_tokenized_de.tsv > parlament_transcriptions_tokenized_de.txt

That's pretty large. Takes about 2 hours on a 1 CPU binder VM, but several minutes on a modern multicore. 

In [ ]:
import fasttext
model = fasttext.train_unsupervised('parlament_transcriptions_tokenized_de.txt', minCount=3, dim=100, epoch=10)
model.save_model('de.fasttext.bin')
print("Done",file=sys.stderr)

We can also just download a prebuilt model instead of waiting.

In [ ]:
! wget https://files.ifi.uzh.ch/cl/siclemat/lehre/hs19/tm/de.fasttext.bin -O de.fasttext.bin

In [ ]:
import fasttext
model = fasttext.load_model('de.fasttext.bin')

In [ ]:
model.words

In [ ]:
model["partei"]

In [ ]:
model.get_nearest_neighbors('verkehr',20)

A is to B, like ? is to C
`model.get_analogies(A,B,C)`

In [ ]:
model.get_analogies("bundesrat","mann","frau")

## Building a mini-corpus

In [ ]:
! grep europa parlament_transcriptions_tokenized_de.txt > europa_de.txt
! ls -lh europa_de.txt

In [ ]:
import fasttext
europa_model = fasttext.train_unsupervised('europa_de.txt',dim=30,minCount=2,maxn=0,ws=10)

In [ ]:
europa_model.get_nearest_neighbors('frieden',20)

# Text classification with fasttext
For text classification, fasttext generate task-specific embeddings. These are dense representations that support the classification task optimally.

Preparing the input file

In [ ]:
! test -e parlament_transcriptions_tokenized_de.tsv || wget https://files.ifi.uzh.ch/cl/siclemat/lehre/hs19/tm/parlament_transcriptions_tokenized_de.tsv -O parlament_transcriptions_tokenized_de.tsv

In [ ]:
%%bash
cut -f 1 <parlament_transcriptions_tokenized_de.tsv | sort | uniq -c |sort -rn 

In [ ]:
%%bash
grep  SozialdemokratischeFraktion < parlament_transcriptions_tokenized_de.tsv \
 | cut -f 1,3 \
 | sed 's/^/__label__/' \
 | shuf --random-source=parlament_transcriptions_tokenized_de.tsv \
 | head -n 2000 > sp_de.tsv

grep  FraktionderSchweizerischenVolkspartei < parlament_transcriptions_tokenized_de.tsv \
 | cut -f 1,3 \
 | sed 's/^/__label__/' \
 | shuf --random-source=parlament_transcriptions_tokenized_de.tsv \
 | head -n 2000 > svp_de.tsv


In [ ]:
! head sp_de.tsv

In [ ]:
%%bash
head -n 1800 < sp_de.tsv > sp_de.train.tsv
tail -n 200 < sp_de.tsv > sp_de.test.tsv
head -n 1800 < svp_de.tsv > svp_de.train.tsv
tail -n 200 < svp_de.tsv > svp_de.test.tsv

cat sp_de.train.tsv svp_de.train.tsv > svpandsp_de.train.tsv
cat sp_de.test.tsv svp_de.test.tsv > svpandsp_de.test.tsv


In [ ]:
! wc -l svpandsp_de.train.tsv  svpandsp_de.test.tsv

In [ ]:
import fasttext
model = fasttext.train_supervised(input="svpandsp_de.train.tsv")

In [ ]:
model.test("svpandsp_de.test.tsv")

Default values: https://fasttext.cc/docs/en/python-module.html#train_supervised-parameters

Take some text from 
https://www.sp-ps.ch/de/kampagnen/unterschriftensammlungen/nein-zum-steuerbonus-fur-reiche-eltern

In [ ]:
model.predict("""Wir sagen NEIN zum Steuerbonus für reiche Eltern ! SVP , FDP und CVP haben beschlossen, die Kinderabzüge bei der Bundessteuer auf 10‘000 Franken zu erhöhen. Was als Familienförderung verkauft wird , ist ein reiner Bonus für Topverdiener . Denn von den Abzügen bei der direkten Bundessteuer profitieren nur jene , die es nicht nötig haben .""".lower())

Take some text from 
https://www.sp-ps.ch/de/kampagnen/unterschriftensammlungen/nein-zum-steuerbonus-fur-reiche-eltern

In [ ]:
model.predict("""Die Menschen in den Grenzregionen leiden unter der brutalen Kriminalität, die wegen der offenen Grenzen fast täglich in die Schweiz schwappt.""".lower())

## Your task: Try to improve the training parameters 
https://fasttext.cc/docs/en/supervised-tutorial.html


In [ ]:
model = fasttext.train_supervised(input='svpandsp_de.train.tsv', autotuneValidationFile='svpandsp_de.test.tsv', autotuneDuration=60)

In [ ]:
model.test("svpandsp_de.test.tsv")

In [ ]:
model.ws,model.lr,model.epoch

# Identifying politicians


In [ ]:
%%bash
cut -f 2 <parlament_transcriptions_tokenized_de.tsv | sort | uniq -c |sort -rn 

In [ ]:
%%bash
grep  WidmerSchlumpfEveline < parlament_transcriptions_tokenized_de.tsv \
 | cut -f 2,3 \
 | sed 's/^/__label__/' \
 | shuf --random-source=parlament_transcriptions_tokenized_de.tsv \
 | head -n 1000 > wse_de.tsv

grep  BlocherChristoph < parlament_transcriptions_tokenized_de.tsv \
 | cut -f 2,3 \
 | sed 's/^/__label__/' \
 | shuf --random-source=parlament_transcriptions_tokenized_de.tsv \
 | head -n 1000 > bc_de.tsv


In [ ]:
%%bash
head -n 900 < wse_de.tsv > wse_de.train.tsv
tail -n 100 < wse_de.tsv > wse_de.test.tsv
head -n 900 < bc_de.tsv > bc_de.train.tsv
tail -n 100 < bc_de.tsv > bc_de.test.tsv

cat wse_de.train.tsv bc_de.train.tsv > wsebc.train.tsv
cat wse_de.test.tsv bc_de.test.tsv > wsebc.test.tsv


In [ ]:
model = fasttext.train_supervised(input="wsebc.train.tsv",epoch=50,dim=10,wordNgrams=1)

In [ ]:
model.test("wsebc.test.tsv")

In [ ]:
!head bc_de.test.tsv

In [ ]:
model.predict("schlägt der mann sämtliche scheiben ein . man weiss nicht , was man mit ihm machen soll . er kommt für zwei tage in den fürsorgerischen freiheitsentzug . er muss entlassen werden , und er macht am dritten tag das gleiche . in den weisungen der strafjustiz heisst es , man müsse ihn irgendwo unterbringen . sie sagen nun , es gebe keine administrative verwahrung ; ich meine ja nicht eine verwahrung für zehn monate")

In [ ]:
!head wse_de.test.tsv

In [ ]:
model.predict("der einlegerschutz seit inkrafttreten des neuen bankeninsolvenzrechts 2004 nicht mehr nur auf spareinlagen beschränkt ist , sondern sämtliche einlagen von kunden umfasst . zudem ist festzuhalten , dass wertschriften , die sich bei der bank ")

## Your turn
What happens if you add a third politician? e.g. LeuenbergerMoritz